# Notebook Overview

In this notebook we will evaluate the change in protein expression dynamics and the frequency of developmental errors induced by losing one of two promoters. Each pair of promoters will be drawn combinatorially from a collection of promoters acting to upregulate the target gene's transcriptional availability, transcript synthesis, and protein synthesis. We will then visualize the resultant change in developmental error frequency under a range of different biosynthesis conditions.

In [ ]:
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt

from promoters.figures.settings import *

### Setting up and running the simulation

First we must import both a `LinearModel` and a `ConditionSimulation` object from the `promoters` package.

In [ ]:
from promoters.models.linear import LinearModel
from promoters.simulation.environment import ConditionSimulation

Next we need to define the activation strengths for each promoter. These are equivalent to $\eta_1$, $\eta_2$, and $\eta_3$ as defined in the manuscript.

In [ ]:
eta = (0.5, 0.5, 0.5)

We then iterate over this ($\eta_1$, $\eta_2$, $\eta_3$) tuple, constructing a model and performing a `ConditionSimulation` for each configuration. As we are only interested in evaluating the error frequencies, we will discard the simulated dynamics and only keep the `simulation.comparisons` instance for each simulation. Note that with $N=5000$ trajectories per condition, this can take a while. We have provided a completed set of simulations in case you wish to skip this step.

In [ ]:
# specify number of trajectories to simulate
N = 50

# iterate over pairs of promoters
comparisons = {}
for i in range(3):

    # define activation strengths
    removed = [0, 0, 0]
    removed[i] = eta[i]

    # define model
    model = LinearModel(g1=0.01, g2=0.001, include_activation=True)
    model.add_promoters(*removed, perturbed=True)

    # run simulation and save comparisons
    simulation = ConditionSimulation(model)
    simulation.run(skwargs=dict(N=N))
    comparisons[i] = simulation.comparisons

### Generating the figure

We will now plot the results as 3x1 grids for each set of metabolic conditions. Each position in the grid will represent the corresponding `threshold_error` - that is the error frequency evaluated for loss of the auxiliary promoter denoted by that position in the grid.

Note that `promoters` uses shorthand notation for each of the metabolic conditions. Specifically:

  * 'normal': normal metabolism
  * 'diabetic': reduced energy metabolism
  * 'hyper_metabolic': elevated energy metabolism

In [ ]:
def plot_heatmap(comparisons, condition):
        
    # assemble matrix
    matrix = np.zeros((3, 1), dtype=float)
    for i, comparison in comparisons.items():
        matrix[i] = comparison[condition].threshold_error

    # plot matrix
    fig, ax = plt.subplots(figsize=(1, 3))
    ax.imshow(matrix, cmap=plt.cm.copper_r, vmin=0, vmax=1)
    ax.set_title('{:s}'.format(condition.upper()))
    ax.set_ylabel('Auxiliary Promoter Target')
    ax.set_xticks([])
    ax.set_yticks([0,1,2])
    ax.set_yticklabels(['Gene','mRNA','Protein'])
    
    return fig

In [ ]:
for condition in ('normal', 'diabetic', 'hyper_metabolic'):
    plot_heatmap(comparisons, condition)

# Completed Simulations from Manuscript

Each of the simulations conducted in support of our manuscript are provided below.

#### Linear model in which auxiliary promoters are removed
``data/simulations/pairwise_simulations/promoters.pkl``

In [ ]:
# specify a path to manuscript data
MANUSCRIPT_DATA_PATH = "../../manuscript/data"

# load repressor simulations
with open(os.path.join(MANUSCRIPT_DATA_PATH, 'simulations/pairwise_simulations/promoters.pkl'), 'rb') as file:
    comparisons = pickle.load(file)

In [ ]:
# plot heatmap for each condition
for condition in ('normal', 'diabetic', 'hyper_metabolic'):
    fig = plot_heatmap(comparisons, condition)